In [29]:
import os
import json

import pandas as pd

In [30]:
FOLDER = '/mnt/hgfs/ssd_tmp/'
h5_filename = os.path.join(FOLDER, 'das_example.h5')

reference_file = 'results_live/das_h5/das_example_ref_fs.json'

with open(reference_file,'r') as f:
  data = json.load(f)
f"{reference_file} has {len(data['refs'])} references"

FileNotFoundError: [Errno 2] No such file or directory: 'results_live/das_h5/das_example_ref_fs.json'

In [ ]:
def create_reference_dataFrame(data):
    '''
        Create a dataframe from the references
    '''
    ref_l = []
    template = data['templates']
    print(template['u'])

    for k,v in data['refs'].items():
        if isinstance(v,list):
            ref_d = {}
            ref_d['name'] = k
            ref_d['key'] = template[v[0][2]]
            ref_d['offset'] = v[1]
            ref_d['bytes'] = v[2]
            ref_d['end'] = ref_d['offset'] + ref_d['bytes'] - 1
            ref_l.append(ref_d)

    # create dataframe sorted by offset and reindexed
    df = pd.DataFrame(ref_l).sort_values('offset').reset_index()
    #
    # add gap column
    df['gap'] = df['offset'] - df.shift(1)['end']

    # return the dataframe
    return df

In [ ]:
ref_df = create_reference_dataFrame(data)
total_data = ref_df.bytes.sum()
maxEnd = ref_df.end.max()
coverage = total_data / maxEnd
print(ref_df.head())
f"Total data: {total_data} Largest end: {maxEnd}  Approximate coverage: {coverage:.2f}"

In [ ]:
#
# calculate the gap between contiguous blocks
#
ref_df['gap'] = ref_df['offset'] - ref_df.shift(1)['end']
pd.set_option('max_colwidth', 90)
ref_df.head(10)

In [ ]:
ref_df.hist(column='bytes')

In [ ]:
ref_df[ref_df.bytes <= 10000].head(30)

In [ ]:
ref_df.bytes.value_counts()

In [ ]:
ref_df.hist(column='gap')

In [ ]:
ref_df[ref_df.gap > 1]